In [1]:
#function:draw scale
def DrawScale(frame,freq):
    if(freq>0):
        #Draw Scale Y
        ScaleRunner = 0
        while(ScaleRunner+freq <int(np.size(frame, 0))):
            cv2.line(frame,(0,ScaleRunner),(10,ScaleRunner),(0,255,255),2)
            cv2.putText(frame,str(ScaleRunner) ,(15,ScaleRunner+10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,255), 2)
            ScaleRunner+=freq
        #Draw Scale X    
        ScaleRunner = 0
        while(ScaleRunner+freq <int(np.size(frame, 1))):
            cv2.line(frame,(ScaleRunner,0),(ScaleRunner,10),(255,255,255),2)
            cv2.putText(frame,str(ScaleRunner) ,(ScaleRunner,25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2)
            ScaleRunner+=freq 
    else:
        cv2.putText(frame,"ScaleError" ,(int(np.size(frame, 0)/2),int(np.size(frame, 0)/2)),cv2.FONT_HERSHEY_SIMPLEX, 4, (0,0,255), 3)

In [2]:
#function:
def DrawContoursCoor(frame,contours,b,g,r):
    i = 0
    for c in contours:
        # calculate moments for each contour
        M = cv2.moments(c)
        # calculate x,y coordinate of center
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        else:
            cX = 0
            cY = 0

        i+=1
        #print('\nX',i,' ',cX)
        #print('Y',i,' ',cY)
        centroid = "X"+str(i)+":"+str(cX)+", Y"+str(i)+":"+str(cY)
        cv2.putText(frame,centroid , (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 1, (b,g,r), 2)

In [3]:
#return list of contour coordinate
def ContoursCoor(contours):
    i = 0
    coor = []
    for c in contours:
        # calculate moments for each contour
        M = cv2.moments(c)
        # calculate x,y coordinate of center
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
        else:
            cX = 0
            cY = 0
        #add list of new coordinate to the previous one
        coor.append([cX,cY])
        
        i+=1

    #print(coor)
    return coor

In [4]:
# import the necessary packages
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np

class CentroidTracker():
    def __init__(self, maxDisappeared=120):
        # initialize the next unique object ID along with two ordered
        # dictionaries used to keep track of mapping a given object
        # ID to its centroid and number of consecutive frames it has
        # been marked as "disappeared", respectively
        self.nextObjectID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()

        # store the number of maximum consecutive frames a given
        # object is allowed to be marked as "disappeared" until we
        # need to deregister the object from tracking
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        # when registering an object we use the next available object
        # ID to store the centroid
        self.objects[self.nextObjectID] = centroid
        self.disappeared[self.nextObjectID] = 0
        self.nextObjectID += 1

    def deregister(self, objectID):
        # to deregister an object ID we delete the object ID from
        # both of our respective dictionaries
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, coor):
        # check to see if the list of input coordinate
        # is empty
        if len(coor) == 0:
            # loop over any existing tracked objects and mark them
            # as disappeared
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID] += 1

                # if we have reached a maximum number of consecutive
                # frames where a given object has been marked as
                # missing, deregister it
                if self.disappeared[objectID] > self.maxDisappeared:
                    self.deregister(objectID)

            # return early as there are no centroids or tracking info
            # to update
            return self.objects

        # initialize an array of input centroids for the current frame
        inputCentroids = np.zeros((len(coor), 2), dtype="int")

        # loop over the bounding box rectangles
        i=0
        while(i<int(len(coor))):
            # use the bounding box coordinates to derive the centroid
            cX = int(coor[i][0])
            cY = int(coor[i][1])
            inputCentroids[i] = (cX, cY)
            i+=1

        # if we are currently not tracking any objects take the input
        # centroids and register each of them
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])

        # otherwise, are are currently tracking objects so we need to
        # try to match the input centroids to existing object
        # centroids
        else:
            # grab the set of object IDs and corresponding centroids
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            # compute the distance between each pair of object
            # centroids and input centroids, respectively -- our
            # goal will be to match an input centroid to an existing
            # object centroid
            D = dist.cdist(np.array(objectCentroids), inputCentroids)

            # in order to perform this matching we must (1) find the
            # smallest value in each row and then (2) sort the row
            # indexes based on their minimum values so that the row
            # with the smallest value as at the *front* of the index
            # list
            rows = D.min(axis=1).argsort()

            # next, we perform a similar process on the columns by
            # finding the smallest value in each column and then
            # sorting using the previously computed row index list
            cols = D.argmin(axis=1)[rows]

            # in order to determine if we need to update, register,
            # or deregister an object we need to keep track of which
            # of the rows and column indexes we have already examined
            usedRows = set()
            usedCols = set()

            # loop over the combination of the (row, column) index
            # tuples
            for (row, col) in zip(rows, cols):
                # if we have already examined either the row or
                # column value before, ignore it
                # val
                if row in usedRows or col in usedCols:
                    continue

                # otherwise, grab the object ID for the current row,
                # set its new centroid, and reset the disappeared
                # counter
                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                # indicate that we have examined each of the row and
                # column indexes, respectively
                usedRows.add(row)
                usedCols.add(col)

            # compute both the row and column index we have NOT yet
            # examined
            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            # in the event that the number of object centroids is
            # equal or greater than the number of input centroids
            # we need to check and see if some of these objects have
            # potentially disappeared
            if D.shape[0] >= D.shape[1]:
                # loop over the unused row indexes
                for row in unusedRows:
                    # grab the object ID for the corresponding row
                    # index and increment the disappeared counter
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    # check to see if the number of consecutive
                    # frames the object has been marked "disappeared"
                    # for warrants deregistering the object
                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)

            # otherwise, if the number of input centroids is greater
            # than the number of existing object centroids we need to
            # register each new input centroid as a trackable object
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        # return the set of trackable objects
        return self.objects

In [18]:
#detect marker from video
import cv2
import numpy as np
from matplotlib import pyplot as plt

ct = CentroidTracker()
#cv2.namedWindow('image',cv2.WINDOW_NORMAL)
#cv2.resizeWindow('image', 1280,720)
#cap = cv2.VideoCapture("C:\\Users\sunny.DESKTOP-QGFGEEK\Desktop\GitClone\\1_camera_gait_analysis\walking_videos\\77marked.avi")
#cap = cv2.VideoCapture("C:\\Users\sunny.DESKTOP-QGFGEEK\Desktop\SGPtraining\\test.mp4")
cap = cv2.VideoCapture("C:\\Users\sunny.DESKTOP-QGFGEEK\Desktop\GitClone\\1_camera_gait_analysis\\tester\\Sunny_Walker3.mp4")
while(cap.isOpened()):
    ret, frame, = cap.read()
    if ret:
        imgray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        imblur1 = cv2.medianBlur(imgray,5)
        imblur2 = cv2.GaussianBlur(imblur1,(5,5),0)
        
        ret,thresh = cv2.threshold(imblur2,190,255,cv2.THRESH_BINARY)

        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        #print("Number of objects found = ", len(contours))
       
        
        cv2.drawContours(frame, contours, -1, (0,0,255), 10)
        

        DrawScale(frame,50)
        DrawContoursCoor(frame,contours,255,255,255)
        
        #contours coordinate
        coor = ContoursCoor(contours)
        #contours tracking 
        objects = ct.update(coor)
        #display tracking
        print(objects)
        for (objectID, centroid) in objects.items():
            # draw both the ID of the object and the centroid of the
            # object on the output frame
            text = "ID {}".format(objectID)
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0),5)
            cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)

        cv2.imshow("frame", frame)
        #cv2.imshow("frame", frame)
        
        key = cv2.waitKey(50)
        if key == 27:
            break
    else: 
        break
cap.release()
cv2.destroyAllWindows()

OrderedDict([(0, array([466, 113])), (1, array([468, 163])), (2, array([0, 0]))])
OrderedDict([(0, array([466, 113])), (1, array([468, 163])), (2, array([0, 0]))])
OrderedDict([(0, array([466, 113])), (1, array([467, 164])), (2, array([0, 0]))])
OrderedDict([(0, array([466, 112])), (1, array([466, 165])), (2, array([0, 0]))])
OrderedDict([(0, array([466, 111])), (1, array([466, 165])), (2, array([0, 0]))])
OrderedDict([(0, array([466, 111])), (1, array([465, 165])), (2, array([0, 0]))])
OrderedDict([(0, array([465, 115])), (1, array([464, 164])), (2, array([0, 0]))])
OrderedDict([(0, array([465, 115])), (1, array([463, 162])), (2, array([0, 0]))])
OrderedDict([(0, array([465, 119])), (1, array([462, 162])), (2, array([0, 0]))])
OrderedDict([(0, array([462, 150])), (1, array([461, 159])), (2, array([0, 0])), (3, array([636, 295])), (4, array([0, 0]))])
OrderedDict([(0, array([464, 127])), (1, array([461, 159])), (2, array([0, 0])), (3, array([630, 294])), (4, array([0, 0]))])
OrderedDic

OrderedDict([(0, array([465, 119])), (1, array([461, 160])), (2, array([437, 286])), (3, array([445, 259])), (4, array([475, 107])), (5, array([441, 316])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 119])), (1, array([461, 160])), (2, array([432, 285])), (3, array([442, 260])), (4, array([475, 107])), (5, array([434, 315])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 119])), (1, array([461, 160])), (2, array([429, 285])), (3, array([438, 260])), (4, array([475, 107])), (5, array([429, 315])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 119])), (1, array([461, 160])), (2, array([424, 285])), (3, array([434, 260])), (4, array([475, 107])), (5, array([423, 315])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 119])), (1, array([

OrderedDict([(0, array([465, 118])), (1, array([468, 169])), (2, array([318, 291])), (3, array([314, 263])), (4, array([467, 104])), (5, array([338, 316])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([468, 169])), (2, array([308, 291])), (3, array([305, 262])), (4, array([467, 104])), (5, array([329, 315])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([468, 169])), (2, array([299, 290])), (3, array([296, 261])), (4, array([467, 104])), (5, array([320, 313])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([468, 169])), (2, array([290, 289])), (3, array([289, 259])), (4, array([467, 104])), (5, array([311, 311])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([

OrderedDict([(0, array([465, 118])), (1, array([468, 165])), (2, array([144, 275])), (3, array([138, 249])), (4, array([467, 103])), (5, array([154, 305])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([468, 166])), (2, array([142, 275])), (3, array([135, 249])), (4, array([467, 103])), (5, array([153, 305])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([468, 166])), (2, array([140, 276])), (3, array([132, 249])), (4, array([467, 103])), (5, array([152, 305])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([468, 166])), (2, array([138, 276])), (3, array([129, 250])), (4, array([467, 103])), (5, array([150, 305])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([

OrderedDict([(0, array([465, 118])), (1, array([469, 166])), (2, array([  1, 263])), (3, array([  1, 238])), (4, array([467, 102])), (5, array([  0, 294])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([469, 166])), (2, array([  1, 263])), (3, array([  1, 238])), (4, array([467, 102])), (5, array([  0, 294])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([469, 166])), (2, array([  1, 263])), (3, array([  1, 238])), (4, array([467, 102])), (5, array([  0, 294])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])
OrderedDict([(0, array([465, 118])), (1, array([469, 166])), (2, array([  1, 263])), (3, array([  1, 238])), (4, array([467, 102])), (5, array([  0, 294])), (6, array([0, 0])), (7, array([0, 0])), (8, array([0, 0])), (9, array([0, 0]))])


====================================================================================
====================================================================================
====================================================================================
====================================================================================
====================================================================================
====================================================================================

In [8]:
#detect marker in image sequences
import cv2
import numpy as np
from matplotlib import pyplot as plt
cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 1280,720)
cap = cv2.VideoCapture("C:\\Users\sunny.DESKTOP-QGFGEEK\Desktop\GitClone\\1_camera_gait_analysis\walking_77_image_sequence\\img (%d).jpg")

while(cap.isOpened()):
    ret, frame, = cap.read()
    if ret:
        imgray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        imblur1 = cv2.medianBlur(imgray,5)
        imblur2 = cv2.GaussianBlur(imblur1,(5,5),0)
        
        ret,thresh = cv2.threshold(imblur2,240,255,cv2.THRESH_BINARY)

        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        #print("Number of objects found = ", len(contours))
        DrawContoursCoor(frame,contours,255,255,255)
        DrawScale(frame,50)
        a = ContoursCoor(contours)
        #print(a)
        #print(len(a))
        
        cv2.drawContours(frame, contours, -1, (0,0,255), 10)
        cv2.imshow("image", frame)
        
        key = cv2.waitKey(30)
        if key == 27:
            break
    else: 
        break
cap.release()
cv2.destroyAllWindows()

In [6]:
#detect marker in webcam
import cv2
import numpy as np
from matplotlib import pyplot as plt
ct = CentroidTracker()
cap = cv2.VideoCapture(0)

while(cap.isOpened()):
    ret, frame, = cap.read()
    if ret:
        imgray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        imblur1 = cv2.medianBlur(imgray,5)
        imblur2 = cv2.GaussianBlur(imblur1,(5,5),0)
        
        ret,thresh = cv2.threshold(imblur2,250,255,cv2.THRESH_BINARY)

        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        #print("Number of objects found = ", len(contours))
    
        #DrawContoursCoor(frame,contours,255,255,255)
        DrawScale(frame,50)
        
        cv2.drawContours(frame, contours, -1, (0,0,255), 10)
        #contours coordinate
        coor = ContoursCoor(contours)
        #contours tracking 
        objects = ct.update(coor)
        #display tracking
        for (objectID, centroid) in objects.items():
            # draw both the ID of the object and the centroid of the
            # object on the output frame
            text = "ID {}".format(objectID)
            cv2.putText(frame, text, (centroid[0] - 10, centroid[1] - 10),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0),2)
            cv2.circle(frame, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
        cv2.imshow("image", frame)
        
        key = cv2.waitKey(30)
        if key == 27:
            break
    else: 
        break
cap.release()
cv2.destroyAllWindows()